In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d sovitrath/diabetic-retinopathy-2015-data-colored-resized

Dataset URL: https://www.kaggle.com/datasets/sovitrath/diabetic-retinopathy-2015-data-colored-resized
License(s): CC0-1.0
 99% 1.93G/1.94G [00:22<00:00, 156MB/s]
100% 1.94G/1.94G [00:22<00:00, 91.3MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/diabetic-retinopathy-2015-data-colored-resized.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import ResNet50

In [ ]:
conv_base = ResNet50(
    weights='imagenet',
    include_top = False,
    input_shape=(224,224,3)
)

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [ ]:
conv_base.trainable = False

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Move validation_split here
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory='/content/colored_images/colored_images',
    subset="training",  # Specify "training" subset
    seed=123,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = train_datagen.flow_from_directory(
    directory='/content/colored_images/colored_images',
    subset="validation",  # Specify "validation" subset
    seed=123,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 28103 images belonging to 5 classes.
Found 7023 images belonging to 5 classes.


In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Assuming your model is already defined and compiled
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
 62/879 [=>............................] - ETA: 1:25:48 - loss: 4.5022 - accuracy: 0.6295

KeyboardInterrupt: 